In [1]:
import pandas as pd
import os
import glob
from datetime import datetime

In [2]:
path = os.getcwd() + "\\Dataset\\"
files = glob.glob(path + "/*precios*")

In [3]:
df_precios = pd.read_excel(files[0], dtype=object ,sheet_name=1)
df_precios1 = pd.read_excel(files[0], sheet_name=0)
df_precios2 = pd.read_csv(files[1],sep=',',encoding='utf-16')
df_precios3 = pd.read_json(files[2])

In [4]:
def unDate(x):
    """read_excel me trae algunas sucursal_id como si fueran datetime. Esta funcion toma elementos, chequea si son datetime y en caso de serlo, los convierte a str con el formato acorde al orden de identificadores de sucursal"""
    if isinstance(x,datetime):
        x=x.strftime("%d-%m-%Y")
    return x

def nasDups(df):
    df = df.dropna()
    df = df.drop_duplicates()
    return df

In [77]:
df_precios.sucursal_id = df_precios['sucursal_id'].apply(unDate)

Vemos que hay nulos en precio y en producto_id.  
Los producto_id están como float, vamos a convertirlos

In [79]:
df_precios.producto_id =  df_precios['producto_id'].apply(lambda x: int(x) if isinstance(x,float) else None)
# df_precios.precio = df_precios.precio.astype(float)

In [112]:
df_precios2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 472131 entries, 0 to 472165
Data columns (total 3 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   precio       472131 non-null  float64
 1   producto_id  472131 non-null  object 
 2   sucursal_id  472131 non-null  object 
dtypes: float64(1), object(2)
memory usage: 14.4+ MB


In [111]:
df_precios2 = nasDups(df_precios2)

In [123]:
def procProd(x):
    if "-" in x:
        x = x.split('-')[2]
    else:
        str(x)

In [134]:
df_precios2.producto_id.apply(lambda x: x.split('-')[2] if '-' in x else None)

0         None
1         None
2         None
3         None
4         None
          ... 
472161    None
472162    None
472163    None
472164    None
472165    None
Name: producto_id, Length: 472131, dtype: object

In [132]:
for item in df_precios2.producto_id:
    if '-' in item:
        print(item.split('-')[2])

2300073000005
2300073000005
2300075000003
2300160000000
2300175000002
2300222000009
2300377000008
2300377000008
2300377000008
2300377000008
2300377000008
2300377000008
2300377000008
2300397000002
2300397000002
2300397000002
2300397000002
2300403000002
2300438000008
2300453000007
2300453000007
2300453000007
2300453000007
2300453000007
2300453000007
2300453000007
2300468000009
2300534000001
2300603000000
2300603000000
2300756000001
2300756000001
2300757000000
2300944000004
2300973000006
2300975000004
2300975000004
2300976000003
2301038000009
2301038000009
2301038000009
2301038000009
2301038000009
2301038000009
2301038000009
2301118000004
2301150000000
2301152000008
2301153000007
2301154000006
2301154000006
2301154000006
2301155000005
2301156000004
2301158000002
2301161000006
2301161000006
2301161000006
2301162000005
2301162000005
2301181000000
2301181000000
2301183000008
2301183000008
2301184000007
2301221000007
2301245000007
2301245000007
2301245000007
2301298000009
2301298000009
230130

In [147]:
for item in df_precios2.producto_id:
    if isinstance(item,str):
        if '-' in item:
            print(item)

10-1-2300073000005
10-1-2300073000005
10-1-2300075000003
10-1-2300160000000
10-1-2300175000002
10-1-2300222000009
10-1-2300377000008
10-1-2300377000008
10-1-2300377000008
10-1-2300377000008
10-1-2300377000008
10-1-2300377000008
10-1-2300377000008
10-1-2300397000002
10-1-2300397000002
10-1-2300397000002
10-1-2300397000002
10-1-2300403000002
10-1-2300438000008
10-1-2300453000007
10-1-2300453000007
10-1-2300453000007
10-1-2300453000007
10-1-2300453000007
10-1-2300453000007
10-1-2300453000007
10-1-2300468000009
10-1-2300534000001
10-1-2300603000000
10-1-2300603000000
10-1-2300756000001
10-1-2300756000001
10-1-2300757000000
10-1-2300944000004
10-1-2300973000006
10-1-2300975000004
10-1-2300975000004
10-1-2300976000003
10-1-2301038000009
10-1-2301038000009
10-1-2301038000009
10-1-2301038000009
10-1-2301038000009
10-1-2301038000009
10-1-2301038000009
10-1-2301118000004
10-1-2301150000000
10-1-2301152000008
10-1-2301153000007
10-1-2301154000006
10-1-2301154000006
10-1-2301154000006
10-1-2301155

In [128]:
def preProc(df):
    df = df.copy()
    df = df.dropna()
    df = df.drop_duplicates()
    return df

In [129]:
df_precios2 = preProc(df_precios2)

In [144]:
count = 0
count_str = 0
for item in df_precios.producto_id:
    if isinstance(item,str):
        if '-' in item:
            count_str +=1
    else:
        count+=1
